In [63]:
import ollama

def generate(prompt):
    # https://github.com/ollama/ollama-python/blob/00eafed0faa5dea6879a8eb3229c7a8f2439abb4/ollama/_types.py#L93
    prompt = f"<s>{prompt.strip()}" #[TOOL_CALLS] "
    # prompt = "<s>" + prompt.strip()
    print(f"PROMPT: ----------------\n{prompt}\n----------------")
    return ollama.generate(
        model = 'llama3.1:latest',
        # system = system,
        # Raw is set to true to feed the question as needed
        raw=True,
        prompt=prompt,
        stream = True,
        # Number of seconds to keep the connection alive
        # keep_alive=60*60,
        options = {
            # 'stop': ['</s>', '}}]'],
            'top_k': 1,
            #'cache': False,
            # 'tfs_z': 2.0,
            'num_ctx': 2048,
            'temperature': 0.,
            'top_p': 0.0
        },
    )


def get_llm_response(prompt, ret_output=True):
    ret = ''
    print('', flush=True)
    for token in generate(prompt):
        print(token['response'], end='', flush=True)
        ret += token['response']
    if ret_output:
        return ret

In [ ]:
class UserPrompt:
    def __init__(self, message, is_lead=False):
        self.turn = 'user'
        self.message = message
        self.is_lead = is_lead
    
    def __str__(self):
        prompt = f'''\n<|start_header_id|>user<|end_header_id|>\n'''
        if self.message:
            prompt += self.message + '\n\n'
        return prompt
        
class AgentPrompt:
    def __init__(self, message='', is_lead=False):
        self.turn = 'agent'
        self.message = message
        if self.message == '':
            self.is_lead = True
        else:
            self.is_lead = is_lead
    
    def __str__(self):
        prompt = f'''\n<|start_header_id|>assistant<|end_header_id|>\n'''
        if self.message:
            prompt += self.message + '\n\n'
        return prompt    


class ToolInfo:
    def __init__(self, fun_hint, func_json):
        self.turn = 'tool'
        self.fun_hint = fun_hint
        self.func_json = func_json

    def __str__(self):
        pass


class System:
    def __init__(self, fun_hint, func_json):
        self.turn = 'tool'
        self.fun_hint = fun_hint
        self.func_json = func_json

    def __str__(self):
        pass

In [ ]:
import datetime

def PROMPT_TEMPLATE():
    current_date = datetime.now()
    formatted_date = current_date.strftime("%d %B %Y")

    prompt = \
f'''<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}'''

    prompt += \
'''You have access to the following functions:
Use the function 'know_anything' to: Know anything after the knowledge cutting date or to know current information
{
    "name": "know_anything",
    "description": "Get information related to anything after your knowledge cutting date",
    "parameters": {
        "query": {
            "param_type": "str",
            "description": "The information that you want to know",
            "required": true
        }
    }
}

Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=know_anything>{"query": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful Assistant.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
What is the current currancy rate for CAD = USD ?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
'''

In [64]:
from duckduckgo_search import DDGS
from trafilatura import fetch_url, extract
import re
import json


def web_search(search_query, get_web_contents=True):
    results = DDGS().text(search_query, max_results=1)
    sdata = []

    for result in results:
        web = fetch_url(result['href'])
        sdata.append({'title': result['title'], 'content': extract(web)[:4000]})

    # results = json.dumps(results, indent=1)
    return sdata

def parse_tool_response(response: str):
    function_regex = r"<function=(\w+)>(.*?)</function>"
    match = re.search(function_regex, response)

    if match:
        function_name, args_string = match.groups()
        try:
            args = json.loads(args_string)
            return {
                "function": function_name,
                "arguments": args,
            }
        except json.JSONDecodeError as error:
            print(f"Error parsing function arguments: {error}")
            return None
    return None

# print(parse_tool_response(response.choices[0].message.content))

* https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#built-in-python-based-tool-calling

* https://github.com/meta-llama/llama-agentic-system/blob/b266c1c62bea3c184a93b1bb295468a41d7b555b/llama_agentic_system/system_prompt.py

In [65]:
prompt = \
'''<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You have access to the following functions:
Use the function 'know_anything' to: Know anything after the knowledge cutting date or to know current information
{
    "name": "know_anything",
    "description": "Get information related to anything after your knowledge cutting date",
    "parameters": {
        "query": {
            "param_type": "str",
            "description": "The information that you want to know",
            "required": true
        }
    }
}

Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=know_anything>{"query": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful Assistant.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
What is the current currancy rate for CAD = USD ?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
'''

output = get_llm_response(prompt)


PROMPT: ----------------
<s><|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You have access to the following functions:
Use the function 'know_anything' to: Know anything after the knowledge cutting date or to know current information
{
    "name": "know_anything",
    "description": "Get information related to anything after your knowledge cutting date",
    "parameters": {
        "query": {
            "param_type": "str",
            "description": "The information that you want to know",
            "required": true
        }
    }
}

Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=know_anything>{"query": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <f

In [67]:
ret = parse_tool_response(output)
if ret is not None:
    web_data = web_search(ret['arguments']['query'])
print(web_data)


[{'title': '1 CAD to USD - Canadian Dollars to US Dollars Exchange Rate - Xe', 'content': '1.00 Canadian Dollar =\n0.72336446 US Dollars\n1 USD = 1.38243 CAD\nXe Currency Converter\n1.00 Canadian Dollar =\n0.72336446 US Dollars\n1 USD = 1.38243 CAD\nCAD | USD |\n---|---|\n1 CAD | 0.723364 USD |\n5 CAD | 3.61682 USD |\n10 CAD | 7.23364 USD |\n25 CAD | 18.0841 USD |\n50 CAD | 36.1682 USD |\n100 CAD | 72.3364 USD |\n500 CAD | 361.682 USD |\n1,000 CAD | 723.364 USD |\n5,000 CAD | 3,616.82 USD |\n10,000 CAD | 7,233.64 USD |\n1 CAD = 0 USD\nLast 30 Days | Last 90 Days | |\n---|---|---|\nHigh These are the highest points the exchange rate has been at in the last 30 and 90-day periods. | 0.73446 | 0.73541 |\nLow These are the lowest points the exchange rate has been at in the last 30 and 90-day periods. | 0.72292 | 0.72292 |\nAverage These are the average exchange rates of these two currencies for the last 30 and 90 days. | 0.73005 | 0.73050 |\nVolatility These percentages show how much the ex

In [73]:
prompt = \
'''<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You have access to the following functions:
Use the function 'know_anything' to: Know anything after the knowledge cutting date or to know current information
{
    "name": "know_anything",
    "description": "Get information related to anything after your knowledge cutting date",
    "parameters": {
        "query": {
            "param_type": "str",
            "description": "The information that you want to know",
            "required": true
        }
    }
}

Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=know_anything>{"query": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful Assistant.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
What is the current currancy rate for CAD = USD ?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
<function=know_anything>{"query": "Current exchange rate CAD to USD"} </function>''' + \
f'''<|start_header_id|>ipython<|end_header_id|>
{web_data}

<|start_header_id|>assistant<|end_header_id|>
'''

output = get_llm_response(prompt)


PROMPT: ----------------
<s><|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 23 Jul 2024

You have access to the following functions:
Use the function 'know_anything' to: Know anything after the knowledge cutting date or to know current information
{
    "name": "know_anything",
    "description": "Get information related to anything after your knowledge cutting date",
    "parameters": {
        "query": {
            "param_type": "str",
            "description": "The information that you want to know",
            "required": true
        }
    }
}

Think very carefully before calling functions.
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=know_anything>{"query": "example_value"}</function>

Reminder:
- If looking for real time information use relevant functions before falling back to brave_search
- Function calls MUST follow the specified 